In [1]:
import time
from collections import deque
from mastodon import Mastodon

In [11]:
mastodon = Mastodon(access_token='YvMBnWgknUL41MI3QjRaOn7goLZ1rRpgyHWZjrxIF9E', api_base_url='https://mastodon.social')
max_nodes = 400
seed_users = ['@2024-elections-AxiosNews@flipboard.com', '@ElectionGraphs@newsie.social', '@politicalwire@politicalwire.com']

In [3]:
def handle_rate_limiting(mastodon):
    remaining_requests = mastodon.ratelimit_remaining
    reset_time = mastodon.ratelimit_reset
    current_time = time.time()

    if remaining_requests == 0 and reset_time:
        sleep_duration = reset_time - current_time + 1
        if sleep_duration > 0:
            print(f"Rate limit reached. Sleeping for {sleep_duration:.2f} seconds.")
            time.sleep(sleep_duration)

In [4]:
queue = deque()
collected_users = set() 
friendship_network = dict()

In [5]:
def handle_rate_limit():
    remaining_requests = mastodon.ratelimit_remaining
    reset_time = mastodon.ratelimit_reset
    current_time = time.time()
    if remaining_requests == 0 and reset_time:
        sleep_duration = reset_time - current_time + 5 # Add 1 second buffer
        if sleep_duration > 0:
            print(f"Rate limit reached. Sleeping for {sleep_duration:.2f} seconds.")
            time.sleep(sleep_duration)

In [6]:
def add_friends_to_queue(user_id, followers, followees):
    
    for user in followers + followees:
        friend_id = user['id']
        friendship_network[user_id].add(friend_id)
        
        if friend_id not in collected_users and len(collected_users) < max_nodes:
            queue.append(friend_id)  
            collected_users.add(friend_id)  
            print("user_added")
            friendship_network[friend_id] = set()

In [13]:
def fetch_followers_followees(account_id, cap=200):
    #handle_rate_limit()
    #print(f"Fetching followers for account {account_id}")
    #followers = []
    #next_page = mastodon.account_followers(account_id, limit=40)
    #while next_page:
    #    handle_rate_limit()
    #    followers.extend(next_page)
    #    next_page = mastodon.fetch_next(next_page)
#
    #handle_rate_limit()
    #followees = []
    #next_page = mastodon.account_following(account_id, limit=80)
    #while next_page:
    #    handle_rate_limit()
    #    followees.extend(next_page)
    #    next_page = mastodon.fetch_next(next_page)

    #return followers, followees
    followers = []
    followees = []
    max_retries = 5
    retries = 0
    while retries < max_retries:
        try:
            handle_rate_limit()
            print(f"Fetching followers for account {account_id}")
            next_page = mastodon.account_followers(account_id, limit=40)
            while next_page and len(followers) < cap:
                handle_rate_limit()
                followers.extend(next_page)
                if len(followers) >= cap:
                    followers = followers[:cap]
                    break
                next_page = mastodon.fetch_next(next_page)
            break  
        except Exception as e:
            retries += 1
            wait_time = (2 ** retries)
            print(f"Error fetching followers for account {account_id}: {e}. Retrying in {wait_time:.2f} seconds.")
            time.sleep(wait_time)
    else:
        print(f"Failed to fetch followers for account {account_id} after {max_retries} retries.")

    retries = 0
    while retries < max_retries:
        try:
            handle_rate_limit()
            print(f"Fetching followees for account {account_id}")
            next_page = mastodon.account_following(account_id, limit=40)
            while next_page and len(followees) < cap:
                handle_rate_limit()
                followees.extend(next_page)
                if len(followees) >= cap:
                    followees = followees[:cap]
                    break
                next_page = mastodon.fetch_next(next_page)
            break  
        except Exception as e:
            retries += 1
            wait_time = (2 ** retries)
            print(f"Error fetching followees for account {account_id}: {e}. Retrying in {wait_time:.2f} seconds.")
            time.sleep(wait_time)
    else:
        print(f"Failed to fetch followees for account {account_id} after {max_retries} retries.")

    print(len(followers), len(followees))
    return followers, followees

In [14]:
queue = deque()
collected_users = set() 
friendship_network = dict()

for seed_user in seed_users:
    account = mastodon.account_search(seed_user)
    if account:
        account_id = account[0]['id']
        queue.append(account_id)
        collected_users.add(account_id)
        friendship_network[account_id] = set()

while queue:
    current_user_id = queue.popleft()
    print(len(queue), len(collected_users))
    followers, followees = fetch_followers_followees(current_user_id)
    add_friends_to_queue(current_user_id, followers, followees)

2 3
Fetching followers for account 111954214376412052
Fetching followees for account 111954214376412052
10 0
10 0
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
{112481031162295043, 110639394489963304, 208169, 110644137799834890, 109502917860191920, 1078390, 112984036654394103, 108222605660848280, 110639387102497435, 112012317272255934}
11 13
Fetching followers for account 109872275163417550
Fetching followees for account 109872275163417550
55 21
55 21
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_added
user_ad

In [16]:
print(f"Total users collected: {len(collected_users)}")
print(f"Total network capacity: {len(list(friendship_network.items()))}")

Total users collected: 400
Total network capacity: 400


In [29]:
t = friendship_network.copy()
t_l = set(t.keys())
for i in t.keys():
    t[i] = t[i].intersection(t_l)

In [32]:
for i in t.keys():
    t[i] = list(t[i])

In [33]:
import json
with open('friendship_network_data.json', 'w') as f:
    json.dump(t, f, indent=4)